## Climatology and Anomaly in Geoscience: Part 2
An example of how Python is used to explore global sea surface temperature (SST).
* Wenchang Yang (wenchang@princeton.edu)
* Behrooz Ferdowsi (ferdowsi@princeton.edu)
* Department of Geosciences, Princeton University
* Junior Colloquium, Nov. 16, 2020


## A short review of the last session
1. `xarray` basic usage: `open_dataset`, `sel`, `isel`, `mean`, `plot`.
2. Sea surface temperature dataset: ERSST5.
3. SST annual climatology and change, monthly climatology.

## Today's plan

1. SST anomaly.
2. El Nino index (Nino3.4) time series.
3. Interactive Q&A.


## Start analysis

In [ ]:
# import and configuration

# xarray is the core package we are going to use
import xarray as xr
import matplotlib.pyplot as plt # we also use pyplot directly in some cases
import os
# some configurations on the default figure output
%config InlineBackend.figure_format ='retina'
plt.rcParams['figure.dpi'] = 120

In [ ]:
# analysis of last session

# sst data file
# Am I running the notebook on Adroit or not?
if os.uname().nodename.startswith('adroit'):
    ifile = '/home/wenchang/JC2020/ersst5_1979-2018.nc'
else:
    ifile = 'ersst5_1979-2018.nc'
print('file to be opened:',ifile)

# open the sst dataset
ds = xr.open_dataset(ifile)

# get the sst DataArray
sst = ds.sst

# 40-year annual mean climatology
sst_clim = sst.mean('time')

# difference between the first and last 10-year climatologies
sst_early = sst.sel(time=slice('1979-01', '1988-12')).mean('time')
sst_late = sst.sel(time=slice('2009-01', '2018-12')).mean('time')
dsst = sst_late - sst_early
dsst.attrs['long_name'] = 'SST change from 1979-1988 to 2009-2018'
dsst.attrs['units'] = '$^\circ$C'

# monthly climatology: the time dimension is now replaced by month
sst_mclim = sst.groupby('time.month').mean('time')

In [ ]:
# Make 12 subplots using a single command.
sst_mclim.plot.contourf(col='month', col_wrap=6, levels=20, 
                        cmap='Spectral_r', center=False)

## Calculate monthly anomaly
Subtract the monthly climatology from the raw SST data.

In [ ]:
ssta = sst.groupby('time.month') - sst_mclim
# monthly climatology is now removed

In [ ]:
ssta

In [ ]:
# The 1997 winter is a big El Nino season
ssta.sel(time=slice('1997-12', '1998-02')).mean('time').plot.contourf(levels=19)

In [ ]:
# The 1998 winter is a big La Nina season
ssta.sel(time=slice('1998-12', '1999-02')).mean('time').plot.contourf(levels=19)

In [ ]:
# The 2015 winter is also a big El Nino season
ssta.sel(time=slice('2015-12', '2016-02')).mean('time').plot.contourf(levels=19)

## Calculate the Nino3.4 index
* SST anomaly averaged over the Nino3.4 region: 170W-120W, 5S-5N
![ENSO](http://www.bom.gov.au/climate/enso/indices/oceanic-indices-map.gif)
http://www.bom.gov.au/climate/enso/indices/oceanic-indices-map.gif

In [ ]:
nino34 = ssta.sel(lon=slice(360-170,360-120),lat=slice(-5,5)).mean(['lon','lat'])
nino34.attrs['long_name'] = 'Nino3.4 index'

In [ ]:
nino34

In [ ]:
nino34.plot()
plt.axvline('2015-12', color='gray', ls='--')
plt.text('2015-12', 2, '2015-12', rotation=-45, color='gray', )
plt.axvline('1997-12', color='gray', ls='--')
plt.text('1997-12', 2, '1997-12', rotation=-45, color='gray', )
plt.axvline('1982-12', color='gray', ls='--')
plt.text('1982-12', 2, '1982-12', rotation=-45, color='gray', )

## Seasonality of El Nino/La Nina

In [ ]:
# compare Nino3.4 index variability for each month
da = nino34.groupby('time.month').std('time') # standard deviation for each month
da = da.roll(month=-5).assign_coords(month=range(6, 18)) # roll the time series to start from Jun
da.plot()
plt.bar(da.month, da.values, color='lightgray')
plt.ylabel('Nino3.4 standard deviation [$^\circ$C]')
plt.xticks(range(6,18), ['Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May'])
plt.grid(True)

December shows the largest variability!

## What's next

* Python resource: https://wy2136.github.io/python.html
* Popular packages: `xarray`, `pandas`, `numpy`, `matplotlib`.
* Machine learning: `sklearn`, `keras`, `pytorch`.
* Try to apply these tools to your project.
* Google first for a question.
* Ask around if google doesn't help.

## Q&A